In [1]:
import pickle
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score, average_precision_score
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier


import networkx as nx
from tqdm import tqdm
import random
import numpy as np
import pickle
import os
import glob
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn


seed = 42
random.seed(seed)
np.random.seed(seed)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:

perplexions_path = '../data/unified_clean_ant_pairs_formated_def.pickle'
test_dataset_path = '../data/ant_test.txt'

with open(perplexions_path, 'rb') as f: #(ребенок, родитель): перплексия
    ppls = pickle.load(f)


ppls_clean = dict()
for item in ppls.items():
    ppls_clean[(item[0][0].split('(')[0].strip(', '), item[0][1].strip(', '))] = item[1]


In [4]:
pairs = []
y_true = []
non_reversed = []
not_found = []
data = []

with open(test_dataset_path, 'r',encoding='utf-8') as f:
    i = 0
    lines = f.readlines()
    for line in lines:
        ex1, ex2, category = line.strip('\n').split('\t')
        s11, v1, s12 = ex1.split(',')
        s21, v2, s22 = ex2.split(',')
        # if s11 == s21 and s12 == s22:
        v1 = v1.strip(' ')
        v2 = v2.strip(' ')
        if category == 'directional_entailment': # child, parent
            data.append((v1, v2, 1))

        elif category == 'directional_non-entailment': # parent, child
            data.append((v1, v2, 0))
        # else:
        #     non_reversed.append((s11, s12, v1, s21, s22, v2, category))

y_true = [elem[2] for elem in data]

In [33]:
len(data)

2930

In [7]:
all_verteces = []
for elem in data:
    all_verteces.append(elem[0])
    all_verteces.append(elem[1])

all_verteces = list(set(all_verteces))

In [8]:
len(all_verteces)

589

In [5]:
G = nx.DiGraph()
for synset in (wn.all_synsets('n')):
    name = synset.name()
    G.add_node(name)
    hyponyms = synset.hyponyms()

    for hypo in hyponyms:
        new_name = hypo.name()
        G.add_node(new_name)
        G.add_edge(name, new_name)

for synset in (wn.all_synsets('v')):
    name = synset.name()
    G.add_node(name)
    hyponyms = synset.hyponyms()

    for hypo in hyponyms:
        new_name = hypo.name()
        G.add_node(new_name)
        G.add_edge(name, new_name)


In [15]:
in_wnet = 0
found = []

for node in all_verteces:
    node = node.replace(' ', '_')
    for i in range(10):
        true_name = f'{node}.v.0{i}'
        if true_name in G.nodes():
            in_wnet += 1
            found.append(node)
            break

In [16]:
in_wnet

7

In [21]:
all_verteces

['listened to',
 'was worn out by',
 'was intrigued by',
 'adhered to',
 'voided',
 'watched',
 'took',
 'was compared to',
 'sharpened',
 'proved',
 'was administered to',
 'brought',
 'asserted',
 'stifled',
 'lengthened',
 'dragged out',
 'capitulate to',
 'reduced',
 'was equal to',
 'focused',
 'affected',
 'knew about',
 'toughened',
 'scattered',
 'honed',
 'took out',
 'mollified',
 'illegalized',
 'whetted',
 'missed',
 'opposed',
 'hindered',
 'unified',
 'was killed by',
 'constricted',
 'judged',
 'departed from',
 'yelled',
 'checked',
 'downloaded',
 'emptied',
 'approached',
 'reassured',
 'revived',
 'refuted',
 'had an opinion of',
 'was rejuvenated by',
 'bothered',
 'saw',
 'decriminalized',
 'was evaluated against',
 'widened',
 'adjusted',
 'divided',
 'acceded to',
 'preserved',
 'praised',
 'participated in',
 'received',
 'smoothed',
 'showed',
 'invalidated',
 'enhanced',
 'were administered to',
 'was revived by',
 'amended',
 'modified',
 'shot down',
 'was a

In [19]:
from nltk.stem import WordNetLemmatizer

In [23]:
lemmatizer = WordNetLemmatizer()

lemmatizer.lemmatize("reduced", pos='v')

'reduce'

In [24]:
in_wnet = 0
found = []

for node in all_verteces:
    node = lemmatizer.lemmatize(node, pos='v')
    node = node.replace(' ', '_')
    for i in range(10):
        true_name = f'{node}.v.0{i}'
        if true_name in G.nodes():
            in_wnet += 1
            found.append(node)
            break

In [25]:
in_wnet

338

In [55]:
both_found = {}

for idx, elem in enumerate(data):
    child = elem[0]
    parent = elem[1]

    #child = lemmatizer.lemmatize(child, pos='v')
    child = child.replace(' ', '_')

    #parent = lemmatizer.lemmatize(parent, pos='v')
    parent = parent.replace(' ', '_')
    

    children = []
    for i in range(10):
        true_name = f'{child}.v.0{i}'
        if true_name in G.nodes():
            children.append(true_name)
    
    parents = []
    for i in range(10):
        true_name = f'{parent}.v.0{i}'
        if true_name in G.nodes():
            parents.append(true_name)

    both_found[idx] = (children, parents)

In [56]:
drop_keys = []
for k, v in both_found.items():
    if (len(v[0]) == 0) or (len(v[1]) == 0):
        drop_keys.append(k)

In [57]:
for k in drop_keys:
    both_found.pop(k)

In [58]:
both_found

{}

In [51]:
paths = []
for k, v in both_found.items():
    children = v[0]
    parents = v[1]

    min_path = 99999999

    for child in children:
        for parent in parents:
            try:
                path = (nx.shortest_path_length(G, parent, child))
            except nx.NetworkXNoPath:
                path = 99999999
            
            if path < min_path:
                min_path = path
    
    paths.append(path)

In [54]:
both_found

{32: (['sedate.v.01'],
  ['take.v.01',
   'take.v.02',
   'take.v.04',
   'take.v.06',
   'take.v.08',
   'take.v.09']),
 33: (['take.v.01',
   'take.v.02',
   'take.v.04',
   'take.v.06',
   'take.v.08',
   'take.v.09'],
  ['sedate.v.01']),
 34: (['calm.v.01', 'calm.v.03'],
  ['take.v.01',
   'take.v.02',
   'take.v.04',
   'take.v.06',
   'take.v.08',
   'take.v.09']),
 35: (['take.v.01',
   'take.v.02',
   'take.v.04',
   'take.v.06',
   'take.v.08',
   'take.v.09'],
  ['calm.v.01', 'calm.v.03']),
 38: (['stimulate.v.01', 'stimulate.v.03', 'stimulate.v.04', 'stimulate.v.06'],
  ['take.v.01',
   'take.v.02',
   'take.v.04',
   'take.v.06',
   'take.v.08',
   'take.v.09']),
 39: (['take.v.01',
   'take.v.02',
   'take.v.04',
   'take.v.06',
   'take.v.08',
   'take.v.09'],
  ['stimulate.v.01', 'stimulate.v.03', 'stimulate.v.04', 'stimulate.v.06']),
 42: (['enliven.v.02'],
  ['take.v.01',
   'take.v.02',
   'take.v.04',
   'take.v.06',
   'take.v.08',
   'take.v.09']),
 43: (['take.v.0

In [53]:
nx.shortest_path_length(G, parent, child)

NetworkXNoPath: No path between overcome.v.02 and battle.v.01.

In [42]:
children, parents

(['battle.v.01'], ['overcome.v.02'])

In [48]:
paths = np.array(paths)

In [49]:
len(paths)

1202

In [50]:
(paths != 99999999).sum()

0